In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
!ls ../

2pcf.readme			   CITAData	cp_CITA2pcf.sh	src
BigMD_2pcfs			   cola_2pcfs	gitpush.sh
cita_cola_compare.worklist.readme  cp_BigMD.sh	haha.sh


In [ ]:
for s_min, s_max, mumax in [[6,40,1.0], [6,40,0.97], [6,40,0.9], [6,40,0.5], 
    [6,20,0.97], [6,40,0.97], [20,40,0.97], 
    [40,60,0.97], 
    [60, 80,0.97], [80, 100,0.97], [100, 150, 0.97]
    ]:
    numsbin=150; nummubin=120;

    fig, axs = plt.subplots(1,3, figsize=(16,6))
    ax1, ax2, ax3 = axs
    Ys = []
    for snapid in [15-0, 15-3, 15-5, 15-7, 15-10, 15-12, 15-15]:    
        # filename, label
        filename = '../CITAData/subsamples/'+\
         'gadget_0%02i'%snapid+'.rat0.0027_ranseed0.shiftr.om0.2951_w-1.0000.radeczw.150s0to150.120mu.2pcf'
        label = r'$z = %.1f'%(redshifts_dict[snapid])+'$'
        ## load in
        data = np.loadtxt(filename); norms = [float(xx) for xx in open(filename, 'r').readline().split()[1:4]]
        ## DD, DR and RR
        DD, DR, RR = [data[:,row].reshape(150,120) for row in [3,4,6]]; 
        DD /= norms[0]; DR /= norms[1]; RR /= norms[2]
        # xi, intximu
        xi = np.divide(DD-2*DR+RR,RR)
        intximu = xi[s_min:s_max+1,:].sum(0)
        # X, Y for plot
        imumax = int(nummubin*mumax)
        #intximu = intximu[:imumax]
        mu = data[:,0]; X = np.ones(nummubin) - mu[:nummubin]; 
        X = X[:imumax]; Y = intximu[:imumax]/intximu[:imumax].sum()*len(intximu[:imumax]); 
        Ys.append([xx for xx in Y])
        # make plot
        ax1.plot(X,Y, marker='o', markersize=1, label=label )
        ax2.plot(X, (Y - np.array(Ys[0])), 
                 marker='o', markersize=1, label=label+ r'  -  $z = 0$' )
        ax3.plot(X, np.divide(Y,  np.array(Ys[0])), 
                 marker='o', markersize=1, label=label+ r'  /  $z = 0$' )
    # axes settings    
    for ax in [ax1, ax2, ax3]:
        ax.grid(); ax.set_xlabel('$1-\mu$', fontsize=18)    
        ax.legend(fontsize=14, frameon=False)
    ax1.set_ylabel(r'$\hat \xi_{\Delta s}(\mu)$', fontsize=22)
    ax2.set_ylabel(r'$\hat \xi_{\Delta s}(\mu) - \hat \xi_{\Delta s}(\mu, z=0)$', fontsize=22)
    ax3.set_ylabel(r'$\hat \xi_{\Delta s}(\mu) / \hat \xi_{\Delta s}(\mu, z=0)$', fontsize=22)
    # fig settings
    fig.suptitle('CITA particles; $s_{\\rm min},\\ s_{\\rm max}$ = '+str(s_min)+', '+str(s_max)+\
     ' Mpc/h; $\mu <= '+str(mumax)+'$.', fontsize=24,); fig.tight_layout(rect=(0, 0, 1, 0.9)); plt.show()
    fig.savefig('s'+str(s_min)+'to'+str(s_max)+'_mumax%.2f'%mumax+'.pdf', format='pdf')